In [3]:
import pandas as pd
import numpy as np
from supervised.automl import AutoML
from dateutil.relativedelta import relativedelta
from sklearn.metrics import roc_auc_score

## DF Original

In [4]:
df_canada = pd.read_csv('canada_updated.csv')
df_canada["date"] = pd.to_datetime(df_canada["date"])
df_canada.head()

,date,QUALITY_FLAG,cid,industry_raw,E_TTM_period_date,E_TTM_ammor_intangibles,E_TTM_asset_writedown,E_TTM_assets_gro_five,E_TTM_capex,E_TTM_cash_acquisitions,...,E_G_ebitda_cov,E_G_ret_on_asset,E_G_ret_on_inv_cap,E_G_net_to_cash,E_G_perm_assets_ratio,return_1q,target_net_income,target_cash_operations,binary_target_net_income,binary_target_cash_operations
0,2002-01-03,True,SP-065996,NaN,2001-10-31,0.000,0.0,0.0,-12.738000,-3.336,...,-165.453488,0.130018,0.101871,-0.068216,0.414230,NaN,NaN,NaN,0,0
1,2002-01-08,True,SP-002396,NaN,2001-09-30,3.078,0.0,0.0,-20.889000,-68.220,...,-2.685925,0.071119,0.067430,-0.004881,0.595752,NaN,NaN,NaN,0,0
2,2002-01-08,True,SP-006704,NaN,2001-09-30,0.000,0.0,0.0,-17.971623,0.000,...,2.262460,-0.069781,-0.039238,-0.045993,0.775432,NaN,NaN,NaN,0,0
3,2002-01-08,True,SP-008644,NaN,2001-09-30,0.000,0.0,0.0,-34.700000,0.000,...,-4.852273,-0.169833,-0.155712,-0.316372,0.773996,NaN,NaN,NaN,0,0
4,2002-01-08,True,SP-013994,NaN,2001-09-30,0.000,0.0,0.0,-1403.000000,-133.000,...,-14.569697,0.109798,0.078497,-0.157934,0.921832,NaN,NaN,NaN,0,0


## Copie pour modification

In [5]:
df_model = df_canada.copy()

df_model['date'] = pd.to_datetime(df_model['date'], errors='coerce')
df_model.sort_values(by=['cid', 'date'], inplace=True)

# Retirer les lignes où Quality_Flag est False
df_model = df_model[df_model['QUALITY_FLAG'] == True]

# (FACULTATIF) Exclure les banques
# df_model = df_model[df_model['industry'] != 'Banks']

## Selection de features

In [6]:
def select_features(
    df,
    include_agro=False,
    include_rgro=False,
    include_tcgro=False,
    include_ratios_assets=False,
    include_ratios_rev=False,
    include_ratios_totcap=False,
    mandatory_cols=None
):
    """
    Sélectionne dynamiquement les colonnes d'un DataFrame en fonction
    des familles de variables explicatives demandées,
    en plaçant d'abord les colonnes obligatoires (mandatory_cols).
    """
    
    # 1) Gérer la liste mandatory_cols (par défaut : vide ou ['cid','date'] selon besoin)
    if mandatory_cols is None:
        mandatory_cols = []
    
    # 2) Définir les "familles" de motifs
    family_patterns = {
        'agro': ['_agro_1q', '_agro_4q'],
        'rgro': ['_rgro_1q', '_rgro_4q'],
        'tcgro': ['_tcgro_1q', '_tcgro_4q'],
        'ratios_assets': ['_on_assets_ratio'],
        'ratios_rev': ['_on_rev_ratio'],
        'ratios_totcap': ['_on_tot_cap_ratio']
    }
    
    # 3) Construire la liste des motifs à inclure
    patterns_to_keep = []
    if include_agro:
        patterns_to_keep += family_patterns['agro']
    if include_rgro:
        patterns_to_keep += family_patterns['rgro']
    if include_tcgro:
        patterns_to_keep += family_patterns['tcgro']
    
    if include_ratios_assets:
        patterns_to_keep += family_patterns['ratios_assets']
    if include_ratios_rev:
        patterns_to_keep += family_patterns['ratios_rev']
    if include_ratios_totcap:
        patterns_to_keep += family_patterns['ratios_totcap']
    
    # 4) Retrouver toutes les colonnes du df qui matchent nos motifs
    matched_cols = set()
    for pat in patterns_to_keep:
        for col in df.columns:
            if pat in col:
                matched_cols.add(col)
    # => matched_cols est un set() de colonnes
    
    # 5) Conserver l'ordre original des colonnes matched, 
    #    en filtrant df.columns dans l'ordre d'origine
    matched_cols_in_order = [c for c in df.columns if c in matched_cols]
    
    # 6) Construire l'ordre final :
    #    - d'abord mandatory_cols (dans l'ordre donné),
    #    - puis les matched_cols (dans l'ordre d'origine)
    #    - attention aux colonnes obligatoires qui n'existent pas, 
    #      ou aux duplications
    #    - on fait donc une intersection + un set() pour éviter 
    #      les collisions.
    
    # Intersection pour ne pas inclure des mandatory inexistantes
    mandatory_cols_in_df = [c for c in mandatory_cols if c in df.columns]
    
    # Puis on concatène en évitant toute duplication
    columns_to_keep_ordered = mandatory_cols_in_df + [
        c for c in matched_cols_in_order if c not in mandatory_cols_in_df
    ]
    
    # 7) Extraire le sous-DataFrame
    df_filtered = df[columns_to_keep_ordered].copy()
    
    # 8) (Optionnel) trier par cid/date si elles sont présentes
    if 'cid' in df_filtered.columns and 'date' in df_filtered.columns:
        df_filtered.sort_values(by=['cid', 'date'], inplace=True)
    
    return df_filtered

## Appel de fonction - Features

### ALL

In [7]:
DF_ALL = select_features(
    df_model,
    include_agro=True,
    include_rgro=True,
    include_tcgro=True,
    include_ratios_assets=True,
    include_ratios_rev=True,
    include_ratios_totcap=True,
    mandatory_cols=['cid', 'date', 'binary_target_net_income']  # je garde la target
)

# Compter le nombre total de NaN dans tout le DataFrame
total_nan = DF_ALL.isna().sum().sum()
print(f"Nombre total de valeurs NaN dans DF_ALL : {total_nan}")

# Retirer les lignes qui contiennent AU MOINS un NaN
DF_ALL.dropna(inplace=True)

# Vérifier à nouveau qu’il n’y a plus de NaN
total_nan_apres = DF_ALL.isna().sum().sum()
print(f"Nombre total de valeurs NaN après suppression : {total_nan_apres}")

print(DF_ALL.shape)

Nombre total de valeurs NaN dans df_model_final : 1360861
Nombre total de valeurs NaN après suppression : 0
(36154, 579)


### Assets

In [ ]:
DF_Assets = select_features(
    df_model,
    include_agro=True,
    include_rgro=False,
    include_tcgro=False,
    include_ratios_assets=True,
    include_ratios_rev=False,
    include_ratios_totcap=False,
    mandatory_cols=['cid', 'date', 'binary_target_net_income']  # je garde la target
)

# Compter le nombre total de NaN dans tout le DataFrame
total_nan = DF_Assets.isna().sum().sum()
print(f"Nombre total de valeurs NaN dans DF_Assets : {total_nan}")

# Retirer les lignes qui contiennent AU MOINS un NaN
DF_Assets.dropna(inplace=True)

# Vérifier à nouveau qu’il n’y a plus de NaN
total_nan_apres = DF_Assets.isna().sum().sum()
print(f"Nombre total de valeurs NaN après suppression : {total_nan_apres}")

print(DF_Assets.shape)

### Revenues

In [ ]:
DF_Revenues = select_features(
    df_model,
    include_agro=False,
    include_rgro=True,
    include_tcgro=False,
    include_ratios_assets=False,
    include_ratios_rev=True,
    include_ratios_totcap=False,
    mandatory_cols=['cid', 'date', 'binary_target_net_income']  # je garde la target
)

# Compter le nombre total de NaN dans tout le DataFrame
total_nan = DF_Revenues.isna().sum().sum()
print(f"Nombre total de valeurs NaN dans DF_Revenues : {total_nan}")

# Retirer les lignes qui contiennent AU MOINS un NaN
DF_Revenues.dropna(inplace=True)

# Vérifier à nouveau qu’il n’y a plus de NaN
total_nan_apres = DF_Revenues.isna().sum().sum()
print(f"Nombre total de valeurs NaN après suppression : {total_nan_apres}")

print(DF_Revenues.shape)

### Capital Moyen

In [ ]:
DF_Capital = select_features(
    df_model,
    include_agro=False,
    include_rgro=False,
    include_tcgro=True,
    include_ratios_assets=False,
    include_ratios_rev=False,
    include_ratios_totcap=True,
    mandatory_cols=['cid', 'date', 'binary_target_net_income']  # je garde la target
)

# Compter le nombre total de NaN dans tout le DataFrame
total_nan = DF_Capital.isna().sum().sum()
print(f"Nombre total de valeurs NaN dans DF_Capital : {total_nan}")

# Retirer les lignes qui contiennent AU MOINS un NaN
DF_Capital.dropna(inplace=True)

# Vérifier à nouveau qu’il n’y a plus de NaN
total_nan_apres = DF_Capital.isna().sum().sum()
print(f"Nombre total de valeurs NaN après suppression : {total_nan_apres}")

print(DF_Capital.shape)

## DF 10 ans

### ALL

In [8]:
# Date la plus récente du DataFrame
max_date = DF_ALL['date'].max()

# Date de coupure (5 ans avant)
cutoff_date = max_date - pd.DateOffset(years=10)

# Filtrer pour ne garder que les 5 dernières années
DF_10Y_all = DF_ALL[DF_ALL['date'] >= cutoff_date].copy()

print(DF_10Y_all['date'].min(), DF_10Y_all['date'].max())
print(DF_10Y_all.shape)

2014-12-12 00:00:00 2024-12-12 00:00:00
(15103, 579)


### Assets

In [ ]:
# Date la plus récente du DataFrame
max_date = DF_Assets['date'].max()

# Date de coupure (5 ans avant)
cutoff_date = max_date - pd.DateOffset(years=10)

# Filtrer pour ne garder que les 5 dernières années
DF_10Y_Assets = DF_Assets[DF_Assets['date'] >= cutoff_date].copy()

print(DF_10Y_Assets['date'].min(), DF_10Y_Assets['date'].max())
print(DF_10Y_Assets.shape)

### Revenues

In [ ]:
# Date la plus récente du DataFrame
max_date = DF_Revenues['date'].max()

# Date de coupure (5 ans avant)
cutoff_date = max_date - pd.DateOffset(years=10)

# Filtrer pour ne garder que les 5 dernières années
DF_10Y_Revenues = DF_Revenues[DF_Revenues['date'] >= cutoff_date].copy()

print(DF_10Y_Revenues['date'].min(), DF_10Y_Revenues['date'].max())
print(DF_10Y_Revenues.shape)

#### Capital Moyen

In [ ]:
# Date la plus récente du DataFrame
max_date = DF_Capital['date'].max()

# Date de coupure (5 ans avant)
cutoff_date = max_date - pd.DateOffset(years=10)

# Filtrer pour ne garder que les 5 dernières années
DF_10Y_Capital = DF_Capital[DF_Capital['date'] >= cutoff_date].copy()

print(DF_10Y_Capital['date'].min(), DF_10Y_Capital['date'].max())
print(DF_10Y_Capital.shape)

## Rolling Window + AutoML

In [3]:
def pipeline_rolling_windows(data, date_col, target_col, train_years, val_years, test_years, label, buffer_months=0):
    """
    Pipeline direct pour la rolling window avec AutoML et cross-validation personnalisée.
    Ajoute les périodes dans le DataFrame final pour validation.
    """
    
    # Conversion de la colonne date
    data[date_col] = pd.to_datetime(data[date_col])
    start_date = data[date_col].min()
    end_date = data[date_col].max()

    predictions_all = []  # Liste pour stocker toutes les prédictions

    while start_date + relativedelta(years=train_years + val_years + test_years) <= end_date:
        # Définir les périodes
        train_end = start_date + relativedelta(years=train_years) - pd.Timedelta(days=1)
        tampon_1_end = train_end + relativedelta(months=buffer_months)
        val_start = tampon_1_end + pd.Timedelta(days=1)
        val_end = val_start + relativedelta(years=val_years) - pd.Timedelta(days=1)
        tampon_2_end = val_end + relativedelta(months=buffer_months)
        test_start = tampon_2_end + pd.Timedelta(days=1)
        test_end = test_start + relativedelta(years=test_years) - pd.Timedelta(days=1)

        # Filtrer les données
        train_data = data.loc[(data[date_col] >= start_date) & (data[date_col] <= train_end)]
        val_data = data.loc[(data[date_col] >= val_start) & (data[date_col] <= val_end)]
        test_data = data.loc[(data[date_col] >= test_start) & (data[date_col] <= test_end)]

        if len(train_data) == 0 or len(val_data) == 0 or len(test_data) == 0:
            print(f"Fenêtre {start_date.year}-{test_end.year} : données insuffisantes, sautée.")
            start_date += relativedelta(years=1)
            continue

        # Configurer et entraîner AutoML
        print(f"Fenêtre {start_date.year}-{test_end.year} : entraînement de AutoML...")
        automl = AutoML(results_path=f"AutoML_{start_date.year}-{test_end.year}_{label}", mode="Perform", algorithms=["Xgboost"], eval_metric="auc")
        custom_cv = [(train_data.index, val_data.index)]
        automl.fit(
            train_data.drop(columns=[target_col, date_col, 'cid']),
            train_data[target_col], cv=custom_cv
        )

        # Prédire sur le test set
        test_preds = test_data[[date_col, target_col]].copy()
        #test_preds["predicted"] = automl.predict_proba(test_data.drop(columns=[target_col, date_col, 'cid']))
        proba = automl.predict_proba(test_data.drop(columns=[target_col, date_col, 'cid']))
        test_preds["prob_down"] = proba[:, 0]  # Probabilité pour la classe 0 (diminution des bénéfices)
        test_preds["prob_up"] = proba[:, 1]  # Probabilité pour la classe 1 (augmentation des bénéfices)
        test_preds["net_prob_score"] = test_preds["prob_up"] - test_preds["prob_down"]
        test_preds["window"] = f"{start_date.year}-{test_end.year}"
        test_preds["cid"] = test_data["cid"].values

        # Ajouter les périodes pour validation
        #test_preds["train_start"] = start_date
        #test_preds["train_end"] = train_end
        #test_preds["tampon_1"] = tampon_1_end
        #test_preds["val_start"] = val_start
        #test_preds["val_end"] = val_end
        #test_preds["tampon_2"] = tampon_2_end
        #test_preds["test_start"] = test_start
        #test_preds["test_end"] = test_end

        # Sauvegarder les prédictions
        predictions_all.append(test_preds)

        # Avancer la fenêtre
        start_date += relativedelta(years=1)

    predictions_df = pd.concat(predictions_all, ignore_index=True)
    return predictions_df

## Appel de fonction - Prédictions

### 10Y ALL

In [4]:
#predictions_df_all = pipeline_rolling_windows(
    #data=DF_10Y_all,
    #date_col="date", 
    #target_col="binary_target_net_income", 
    #train_years=5, 
    #val_years=1, 
    #test_years=1,
    #label="all", 
    #buffer_months=1
#)

#predictions_df_all.to_csv("df10Y_NetInc_Pred_all.csv", index=False)

IndentationError: unexpected indent (2583970802.py, line 2)

### 10Y Assets

In [ ]:
predictions_df_assets = pipeline_rolling_windows(
    data=DF_10Y_Assets,
    date_col="date", 
    target_col="binary_target_net_income", 
    train_years=5, 
    val_years=1, 
    test_years=1,
    label="assets", 
    buffer_months=1
)

predictions_df_assets.to_csv("df10Y_NetInc_Pred_assets.csv", index=False)

### 10Y Revenues

In [ ]:
predictions_df_revenues = pipeline_rolling_windows(
    data=DF_10Y_Revenues,
    date_col="date", 
    target_col="binary_target_net_income", 
    train_years=5, 
    val_years=1, 
    test_years=1,
    label="Revenue", 
    buffer_months=1
)

predictions_df_revenues.to_csv("df10Y_NetInc_Pred_revenues.csv", index=False)

### 10Y Capital

In [ ]:
predictions_df_capital = pipeline_rolling_windows(
    data=DF_10Y_Capital,
    date_col="date", 
    target_col="binary_target_net_income", 
    train_years=5, 
    val_years=1, 
    test_years=1,
    label="capital", 
    buffer_months=1
)

predictions_df_capital.to_csv("df10Y_NetInc_Pred_capital.csv", index=False)

## AUC - Overall

In [12]:
#test_auc_net = roc_auc_score(predictions_df["binary_target_net_income"], predictions_df["net_prob_score"])
#test_auc_up = roc_auc_score(predictions_df["binary_target_net_income"], predictions_df["prob_up"])
#test_auc_down = roc_auc_score(predictions_df["binary_target_net_income"], predictions_df["prob_down"])
#test_auc_net, test_auc_up, test_auc_down

(0.6703369325280739, 0.6703368649485612, 0.3296630674719261)

## Grouped AUCs

In [13]:
def compute_auc_by_groups(
    predictions_df: pd.DataFrame, 
    df_canada: pd.DataFrame,
    group_cols: list,
    cid_col: str = "cid",
    date_col: str = "date",
    target_col: str = "binary_target_net_income",
    prob_col: str = "prob_up",
    out_csv: str = None
):
    """
    Calcule l'AUC par groupes (selon group_cols) et le nombre de lignes dans chaque groupe.
    
    Arguments :
    -----------
    - predictions_df : DataFrame contenant au moins [cid_col, date_col, target_col, prob_col]
    - df_canada : DataFrame contenant au moins [cid_col, date_col, 'return_1q', 'industry_raw', etc.]
    - group_cols : liste des colonnes sur lesquelles faire le groupby (ex: ["year", "industry_raw"])
    - cid_col : nom de la colonne identifiant l'entreprise (défaut: "cid")
    - date_col : nom de la colonne date (défaut: "date")
    - target_col : nom de la colonne cible binaire (ex: 0/1) (défaut: "binary_target_net_income")
    - prob_col : nom de la colonne contenant la probabilité pour la classe positive (défaut: "prob_up")
    - out_csv : chemin vers le fichier CSV de sortie (défaut: None, ne pas exporter)

    Retour :
    --------
    - final_df : DataFrame avec AUC et Count pour chaque groupe
    """

    # 1) Fusionner les dataframes (left merge pour conserver toutes les prédictions)
    df_merged = predictions_df.merge(
        df_canada[[cid_col, date_col, 'return_1q', 'industry_raw']],  # adapt si tu veux plus/moins de colonnes
        on=[cid_col, date_col],
        how='left'
    )

    # 2) Si "year" est dans group_cols et pas encore créée, on l'extrait du date_col
    if "year" in group_cols and "year" not in df_merged.columns:
        df_merged["year"] = pd.to_datetime(df_merged[date_col]).dt.year

    # -- Fonction interne pour calculer l’AUC sur un sous-groupe
    def group_auc(sub_df):
        unique_targets = sub_df[target_col].unique()
        # Gérer le cas où il n’y a qu’une seule classe (roc_auc_score plantait sinon)
        if len(unique_targets) < 2:
            return float("nan")
        return roc_auc_score(sub_df[target_col], sub_df[prob_col])

    # 3) Calculer l’AUC par groupe
    grouped_aucs = (
        df_merged
        .groupby(group_cols)
        .apply(group_auc)
        .reset_index(name="AUC")
    )

    # 4) Calculer le nombre d’observations par groupe (ou nombre de cid uniques, selon besoin)
    #    Ici : on compte simplement le nombre de lignes
    grouped_counts = (
        df_merged
        .groupby(group_cols)[target_col]  # ou [cid_col] si tu veux le nombre d'entreprises distinctes
        .count()
        .reset_index(name="Count")
    )

    # 5) Fusionner AUC et Count
    final_df = pd.merge(grouped_aucs, grouped_counts, on=group_cols, how="left")

    # Optionnel : trier le résultat
    final_df = final_df.sort_values(group_cols)

    # Optionnel : exporter en CSV
    if out_csv is not None:
        final_df.to_csv(out_csv, index=False)

    return final_df

## Appel de Fonction - Grouped AUCs

### ALL

In [14]:
#compute_auc_by_groups(
    #predictions_df=predictions_df_all,
    #df_canada=df_canada,
    #group_cols=["year"],
    #out_csv="DF10Y_NetInc_AUC_Year_all.csv"
#)

/var/folders/ns/nlrkcsjd7j70p9jvf2jl5t4c0000gn/T/ipykernel_1371/2025299659.py:53: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.


,year,AUC,Count
0,2021,0.707484,1414
1,2022,0.707988,1491
2,2023,0.744579,1457
3,2024,NaN,1386


### Assets

In [ ]:
compute_auc_by_groups(
    predictions_df=predictions_df_assets,
    df_canada=df_canada,
    group_cols=["year"],
    out_csv="DF10Y_NetInc_AUC_Year_assets.csv"
)

### Revenues

In [ ]:
compute_auc_by_groups(
    predictions_df=predictions_df_revenues,
    df_canada=df_canada,
    group_cols=["year"],
    out_csv="DF10Y_NetInc_AUC_Year_revenues.csv"
)

### Capital

In [ ]:
compute_auc_by_groups(
    predictions_df=predictions_df_capital,
    df_canada=df_canada,
    group_cols=["year"],
    out_csv="DF10Y_NetInc_AUC_Year_capital.csv"
)

## Création de Portefeuille + Annual Weighted Returns

In [15]:
def create_weighted_portfolios(predictions_df, df_canada, proba_col, return_col, lower_threshold=0.4, upper_threshold=0.6):
    """
    Crée un portefeuille pondéré basé sur les prédictions et calcule les rendements pondérés par année,
    en attribuant des poids positifs aux positions longues et négatifs aux positions courtes, avec une somme neutre.
    
    Args:
    - predictions_df (pd.DataFrame): DataFrame contenant les prédictions et les identifiants 'cid'.
    - df_canada (pd.DataFrame): DataFrame contenant les rendements futurs et les identifiants 'cid'.
    - return_col (str): Nom de la colonne des rendements futurs dans df_canada.
    - lower_threshold (float): Seuil inférieur pour les positions courtes.
    - upper_threshold (float): Seuil supérieur pour les positions longues.
    
    Returns:
    - result_df (pd.DataFrame): DataFrame contenant les rendements pondérés des portefeuilles par année.
    """
    # Assure que les colonnes 'date' sont au bon format datetime dans les deux DataFrames
    predictions_df['date'] = pd.to_datetime(predictions_df['date'])
    df_canada['date'] = pd.to_datetime(df_canada['date'])
        
    
    # Joindre les deux DataFrames sur 'cid' et 'date'
    merged_df = predictions_df.merge(df_canada[['cid', 'date', return_col]], on=['cid', 'date'], how='left')
    
    # Extraire l'année à partir de la colonne 'date'
    merged_df['year'] = merged_df['date'].dt.year
    
    # Retirer les lignes avec des valeurs manquantes
    merged_df.dropna(inplace=True)
    
    # Initialiser une liste pour stocker les résultats
    results = []

    # Grouper par année
    for year, group in merged_df.groupby('year'):
        # Sélectionner les positions longues et courtes selon les seuils
        selected_long = group[group[proba_col] > upper_threshold]  # Positions longues
        selected_short = group[group[proba_col] < lower_threshold]  # Positions courtes
        
        n_long = len(selected_long)
        n_short = len(selected_short)
        
        if n_long > 0 or n_short > 0:
            # Attribuer des poids aux positions longues et courtes
            if n_long > 0:
                selected_long.loc[:, 'weight'] = 1 / n_long  # Poids positifs pour les positions longues
            if n_short > 0:
                selected_short.loc[:, 'weight'] = -1 / n_short  # Poids négatifs pour les positions courtes
            
            # Combiner les deux DataFrames
            selected = pd.concat([selected_long, selected_short], ignore_index=True)
            
            # Calculer le rendement pondéré du portefeuille (somme des rendements pondérés)
            weighted_return = (selected['weight'] * selected[return_col] * 100).sum()
            
            # Ajouter le résultat à la liste
            results.append({'year': year, 'weighted_return': weighted_return})
        else:
            # Si aucune action ne respecte les seuils, le rendement est NaN
            results.append({'year': year, 'weighted_return': float('nan')})

    # Convertir les résultats en DataFrame
    result_df = pd.DataFrame(results)
    
    return result_df

## Appel de fonction rendement annuel

### ALL

In [16]:
#ret_net_income_all = create_weighted_portfolios(predictions_df_all, df_canada, 'prob_up', 'return_1q', lower_threshold=0.5, upper_threshold=0.5)

#ret_net_income.to_csv("returns_net_income_0.5_0.5.csv", index=False)

#ret_net_income_all

/var/folders/ns/nlrkcsjd7j70p9jvf2jl5t4c0000gn/T/ipykernel_1371/444120012.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/var/folders/ns/nlrkcsjd7j70p9jvf2jl5t4c0000gn/T/ipykernel_1371/444120012.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/var/folders/ns/nlrkcsjd7j70p9jvf2jl5t4c0000gn/T/ipykernel_1371/444120012.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

,year,weighted_return
0,2021,4.214508
1,2022,0.797524
2,2023,0.032612
3,2024,-2.644895


In [17]:
#returns_net_income_all = create_weighted_portfolios(predictions_df_all, df_canada, 'prob_up', 'return_1q', lower_threshold=0.4, upper_threshold=0.6)

#returns_net_income.to_csv("returns_net_income_0.4_0.6.csv", index=False)

#returns_net_income_all

/var/folders/ns/nlrkcsjd7j70p9jvf2jl5t4c0000gn/T/ipykernel_1371/444120012.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/var/folders/ns/nlrkcsjd7j70p9jvf2jl5t4c0000gn/T/ipykernel_1371/444120012.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/var/folders/ns/nlrkcsjd7j70p9jvf2jl5t4c0000gn/T/ipykernel_1371/444120012.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

,year,weighted_return
0,2021,4.378128
1,2022,0.891550
2,2023,0.051322
3,2024,-2.973748


## Rendement par Décile

In [18]:
def compute_decile_returns(
    predictions_df: pd.DataFrame,
    df_canada: pd.DataFrame,
    proba_col: str,
    return_col: str,
    aggregator: str = "mean",        # "mean" ou "median"
    decile_weights: list = None,     # liste de 10 poids ou None
    csv_path: str = None             # chemin CSV ou None
) -> pd.DataFrame:
    """
    Calcule le rendement par décile (du plus faible au plus élevé) pour chaque année.
    - Découpage en déciles via pd.qcut (q=10).
    - Agrégation des rendements par décile via un 'aggregator' (mean ou median).
    - Optionnellement, applique un vecteur de poids (decile_weights) pour calculer
      un rendement global (somme pondérée des déciles).

    Paramètres
    ----------
    predictions_df : pd.DataFrame
        Contient les prédictions (probabilités) et les colonnes 'cid', 'date'.
        Doit inclure la colonne `proba_col`.
    df_canada : pd.DataFrame
        Contient la colonne `return_col` ainsi que 'cid', 'date'.
    proba_col : str
        Nom de la colonne dans predictions_df représentant la probabilité ou le score.
    return_col : str
        Nom de la colonne dans df_canada qui contient le rendement (ou variation).
    aggregator : {"mean", "median"}, défaut = "mean"
        Choix entre la moyenne ou la médiane comme statistique de rendement pour le décile.
    decile_weights : list of float (length 10) ou None
        Si fourni, liste de 10 poids (un pour chaque décile de 0 à 9).
        Permet de calculer une colonne "portfolio_return" = somme(décile_i * poids_i).
    csv_path : str ou None
        Si un chemin est fourni, le DataFrame final sera sauvegardé en CSV.

    Retour
    ------
    df_final : pd.DataFrame
        DataFrame pivoté avec:
        - 1 ligne par année
        - 10 colonnes (D1 à D10) indiquant le rendement agrégé de chaque décile
        - Si decile_weights est fourni, une colonne supplémentaire 'portfolio_return'
          représentant la somme pondérée des rendements déciles.
    """

    # --- 1) Merge sur cid + date
    merged_df = predictions_df.merge(
        df_canada[['cid', 'date', return_col]],
        on=['cid', 'date'],
        how='left'
    )
    
    # --- 2) Extraire l'année
    merged_df['year'] = merged_df['date'].dt.year
    
    # --- 3) Nettoyage éventuel
    merged_df.dropna(subset=[proba_col, return_col, 'year'], inplace=True)
    
    # --- 4) Choix de la fonction d'agrégation
    if aggregator == "mean":
        aggregator_func = np.mean
    elif aggregator == "median":
        aggregator_func = np.median
    else:
        raise ValueError("aggregator must be 'mean' or 'median'.")
    
    # --- 5) Préparer une liste pour stocker (year, decile_bin, agg_return)
    decile_results = []
    
    for year, group in merged_df.groupby('year'):
        # On doit découper en 10 déciles
        group = group.copy()
        
        # Astuce: labels=False renvoie des labels 0..9
        group['decile_bin'] = pd.qcut(group[proba_col], q=10, labels=False)
        
        # Calculer l'agrégat (mean ou median) du rendement dans chaque décile
        for decile_id, subdf in group.groupby('decile_bin'):
            if pd.isnull(decile_id):
                continue
            
            decile_id = int(decile_id)
            # Rendement agrégé (en % si on veut)
            decile_value = aggregator_func(subdf[return_col]) * 100.0
            
            decile_results.append({
                'year': year,
                'decile': decile_id,
                'decile_return': decile_value
            })
    
    # --- 6) On transforme la liste en DataFrame + pivot
    decile_df = pd.DataFrame(decile_results)
    df_pivot = decile_df.pivot(index='year', columns='decile', values='decile_return')
    
    # Renommer les colonnes pour D1..D10
    # Attention : decile_id = 0 correspond au 1er décile => D1
    df_pivot.columns = [f"D{i+1}" for i in df_pivot.columns]
    
    # --- 7) Si on a des poids, on calcule la somme pondérée
    if decile_weights is not None:
        if len(decile_weights) != 10:
            raise ValueError("decile_weights must be a list of length 10.")
        
        # Multiplier chaque colonne D1..D10 par le poids correspondant, puis sommer
        # Astuce : df_pivot.iloc[:, 0:10] prend les 10 premières colonnes => D1..D10
        # On peut ensuite .mul(decile_weights, axis='columns') pour multiplier par le vecteur
        # et .sum(axis=1) pour obtenir la somme par année
        # (on suppose que l'ordre des colonnes est D1..D10).
        
        weighted_sum = df_pivot.iloc[:, 0:10].mul(decile_weights, axis='columns').sum(axis=1)
        
        # On l'ajoute comme nouvelle colonne
        df_pivot['portfolio_return'] = weighted_sum
    
    # --- 8) On remet l'index 'year' comme colonne
    df_final = df_pivot.reset_index()
    
    # --- 9) Sauvegarde CSV si demandé
    if csv_path is not None:
        df_final.to_csv(csv_path, index=False)
    
    return df_final

## Appel de fonction - Rendement par Décile

## Decile Weights : [-0.25, -0.25, -0.25, -0.25, 0, 0, 0.25, 0.25, 0.25, 0.25]

### ALL

In [19]:
#df_deciles_all = compute_decile_returns(
    #predictions_df=predictions_df_all,
    #df_canada=df_canada,
    #proba_col='prob_up',
    #return_col='return_1q',
    #aggregator='mean',
    #decile_weights=[-0.25, -0.25, -0.25, -0.25, 0, 0, 0.25, 0.25, 0.25, 0.25],
    #csv_path='DF10Y_NetInc_decile_returns_all.csv'
#)

#df_deciles_all.head()

,year,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,portfolio_return
0,2021,4.606986,2.113403,4.389568,5.358879,8.193158,5.397031,7.390472,4.201719,9.778015,15.064996,4.991591
1,2022,2.671292,0.847816,-3.298129,-3.150311,-3.835077,-3.998812,-3.531703,-4.570153,-0.614925,1.762166,-1.006321
2,2023,0.916306,0.946266,0.816035,4.053523,3.450989,3.270271,1.951572,6.181214,0.803168,0.962816,0.791659
3,2024,4.630117,3.786070,8.280142,5.230720,4.432093,4.134712,4.108770,2.093775,0.884630,3.045103,-2.948693


### Assets

In [ ]:
df_deciles_assets = compute_decile_returns(
    predictions_df=predictions_df_assets,
    df_canada=df_canada,
    proba_col='prob_up',
    return_col='return_1q',
    aggregator='mean',
    decile_weights=[-0.25, -0.25, -0.25, -0.25, 0, 0, 0.25, 0.25, 0.25, 0.25],
    csv_path='DF10Y_NetInc_decile_returns_assets.csv'
)

df_deciles_assets.head()

### Revenues

In [ ]:
df_deciles_revenues = compute_decile_returns(
    predictions_df=predictions_df_revenues,
    df_canada=df_canada,
    proba_col='prob_up',
    return_col='return_1q',
    aggregator='mean',
    decile_weights=[-0.25, -0.25, -0.25, -0.25, 0, 0, 0.25, 0.25, 0.25, 0.25],
    csv_path='DF10Y_NetInc_decile_returns_revenues.csv'
)

df_deciles_revenues.head()

### Capital

In [ ]:
df_deciles_capital = compute_decile_returns(
    predictions_df=predictions_df_capital,
    df_canada=df_canada,
    proba_col='prob_up',
    return_col='return_1q',
    aggregator='mean',
    decile_weights=[-0.25, -0.25, -0.25, -0.25, 0, 0, 0.25, 0.25, 0.25, 0.25],
    csv_path='DF10Y_NetInc_decile_returns_capital.csv'
)

df_deciles_revenues.head()

## Decile Weights : [-0.20, -0.20, -0.20, -0.20, -0.20, 0.20, 0.20, 0.20, 0.20, 0.20]

## Preuve CSV Rendement par Décile

# --- 1) Merge sur cid + date
merged_df = predictions_df.merge(
    df_canada[['cid', 'date', 'return_1q']],
    on=['cid', 'date'],
    how='left'
    )

# --- 2) Extraire l'année
merged_df['year'] = merged_df['date'].dt.year

# Filtrer pour l'année 2023
year = 2023
year_df = merged_df[merged_df['year'] == year].copy()

# Attribuer les déciles pour cette année
year_df['decile_bin'] = pd.qcut(year_df['prob_up'], q=10, labels=False)

# Liste pour stocker chaque subdf avec une colonne indiquant le décile
list_of_subdfs = []

# Itérer sur chaque décile et collecter les sous-ensembles
for decile_id, subdf in year_df.groupby('decile_bin'):
    # Ajouter une colonne indiquant le décile courant
    subdf = subdf.copy()
    subdf['decile_id'] = decile_id
    list_of_subdfs.append(subdf)

# Concaténer tous les subdf en un seul DataFrame
all_subdfs_df = pd.concat(list_of_subdfs, ignore_index=True)

# Exporter vers CSV
#all_subdfs_df.to_csv("subdfs_2023.csv", index=False)